In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='9fe28646-ea2d-4be4-9c55-76c95be04bc4', project_access_token='p-ce641907fd78abfc6af00ec243c63be6c416f7f2')
pc = project.project_context


In [2]:
import pandas as pd
import numpy as np
from project_lib import Project
!pip install holidays
import holidays
import matplotlib.pyplot as plt
import seaborn as sns
project = Project(project_id='9fe28646-ea2d-4be4-9c55-76c95be04bc4', project_access_token='p-ce641907fd78abfc6af00ec243c63be6c416f7f2')
pc = project.project_context

In [3]:
df_load_weather = pd.read_csv(project.get_file("hourly_load_actual_weather_utc_stations_flat.csv"))
df_load_weather["Datetime_UTC"] = pd.to_datetime(df_load_weather["Datetime_UTC"], utc=True)
df_load_weather["Datetime_EST"] = df_load_weather["Datetime_UTC"].dt.tz_convert("US/Eastern")
df_load_weather

,Datetime_UTC,Datetime_EST,MWH_actual,DBWind_AVL,DBDryBulb_AVL,DBWetBulb_AVL,DBRH_AVL,DBSky_AVL,DBWind_EWN,DBDryBulb_EWN,...,DBWind_POB,DBDryBulb_POB,DBWetBulb_POB,DBRH_POB,DBSky_POB,DBWind_RDU,DBDryBulb_RDU,DBWetBulb_RDU,DBRH_RDU,DBSky_RDU
0,2016-01-01 06:00:00+00:00,2016-01-01 01:00:00-05:00,5324.4385,16.0,42.0,40.0,82.0,0.0,6.0,57.0,...,9.0,57.0,56.0,93.0,100.0,5.0,50.0,47.0,77.0,100.0
1,2016-01-01 07:00:00+00:00,2016-01-01 02:00:00-05:00,5137.2178,10.0,41.0,39.0,82.0,0.0,9.0,57.0,...,10.0,54.0,52.0,83.0,100.0,5.0,49.0,46.0,77.0,100.0
2,2016-01-01 08:00:00+00:00,2016-01-01 03:00:00-05:00,4970.3784,9.0,41.0,39.0,79.0,0.0,9.0,57.0,...,5.0,52.0,49.0,78.0,100.0,5.0,48.0,44.0,74.0,80.0
3,2016-01-01 09:00:00+00:00,2016-01-01 04:00:00-05:00,4881.2212,11.0,41.0,39.0,79.0,0.0,8.0,56.0,...,10.0,51.0,48.0,76.0,100.0,3.0,47.0,44.0,74.0,80.0
4,2016-01-01 10:00:00+00:00,2016-01-01 05:00:00-05:00,4916.1343,9.0,40.0,38.0,83.0,0.0,7.0,56.0,...,7.0,51.0,47.0,75.0,80.0,6.0,47.0,43.0,71.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50415,2021-10-01 21:00:00+00:00,2021-10-01 17:00:00-04:00,8506.2471,8.0,73.0,66.0,66.0,50.0,0.0,77.0,...,3.0,79.0,66.0,50.0,20.0,5.0,79.0,66.0,50.0,80.0
50416,2021-10-01 22:00:00+00:00,2021-10-01 18:00:00-04:00,8439.3682,7.0,73.0,66.0,66.0,0.0,5.0,74.0,...,0.0,78.0,66.0,52.0,0.0,6.0,79.0,66.0,50.0,50.0
50417,2021-10-01 23:00:00+00:00,2021-10-01 19:00:00-04:00,8119.7773,3.0,71.0,66.0,73.0,80.0,5.0,71.0,...,0.0,71.0,66.0,73.0,0.0,0.0,73.0,66.0,66.0,80.0
50418,2021-10-02 00:00:00+00:00,2021-10-01 20:00:00-04:00,7875.9663,3.0,68.0,64.0,78.0,20.0,5.0,69.0,...,0.0,67.0,65.0,87.0,0.0,0.0,71.0,65.0,70.0,80.0


In [4]:
def season_calc(month):
    """defining season based on analysis of historical peak load data and temperatures at times of peaks""";
    if month in [6,7,8,9]:
        return "summer"
        
    if month in [5,10]:
        return "t1"
        
    if month in [4,11]:
        return "t2"
        
    else:
        return "winter"

In [5]:
df = df_load_weather
weekdays = {2:'Monday', 3:'Tuesday', 4:'Wednesday', 5: 'Thursday', 6: 'Friday', 7:'Saturday', 1:'Sunday'}
df['Date'] = pd.to_datetime(df.Datetime_EST.dt.date)
df['year'] = df.Datetime_EST.dt.year
df['month'] = df.Datetime_EST.dt.month
df['day'] = df.Datetime_EST.dt.day
df['hour'] = df.Datetime_EST.dt.hour
df['weekday'] = df.Datetime_EST.dt.day_name()
df['season'] = df.Datetime_EST.dt.month.apply(season_calc)
df.head()

,Datetime_UTC,Datetime_EST,MWH_actual,DBWind_AVL,DBDryBulb_AVL,DBWetBulb_AVL,DBRH_AVL,DBSky_AVL,DBWind_EWN,DBDryBulb_EWN,...,DBWetBulb_RDU,DBRH_RDU,DBSky_RDU,Date,year,month,day,hour,weekday,season
0,2016-01-01 06:00:00+00:00,2016-01-01 01:00:00-05:00,5324.4385,16.0,42.0,40.0,82.0,0.0,6.0,57.0,...,47.0,77.0,100.0,2016-01-01,2016,1,1,1,Friday,winter
1,2016-01-01 07:00:00+00:00,2016-01-01 02:00:00-05:00,5137.2178,10.0,41.0,39.0,82.0,0.0,9.0,57.0,...,46.0,77.0,100.0,2016-01-01,2016,1,1,2,Friday,winter
2,2016-01-01 08:00:00+00:00,2016-01-01 03:00:00-05:00,4970.3784,9.0,41.0,39.0,79.0,0.0,9.0,57.0,...,44.0,74.0,80.0,2016-01-01,2016,1,1,3,Friday,winter
3,2016-01-01 09:00:00+00:00,2016-01-01 04:00:00-05:00,4881.2212,11.0,41.0,39.0,79.0,0.0,8.0,56.0,...,44.0,74.0,80.0,2016-01-01,2016,1,1,4,Friday,winter
4,2016-01-01 10:00:00+00:00,2016-01-01 05:00:00-05:00,4916.1343,9.0,40.0,38.0,83.0,0.0,7.0,56.0,...,43.0,71.0,100.0,2016-01-01,2016,1,1,5,Friday,winter


In [6]:
us_holidays = holidays.US()
df['EST_Holiday']= df['Datetime_EST'].apply( lambda x: (x in us_holidays)+0)

#adding in another column which indicates 'non-working' days which include weekends and holidays
df['non_working'] = df.apply(lambda x: 1 if \
                                         ((x['EST_Holiday'] == 1) or (x['weekday'] in ['Saturday', 'Sunday']))
                                         else 0, axis = 1)

df.head()

,Datetime_UTC,Datetime_EST,MWH_actual,DBWind_AVL,DBDryBulb_AVL,DBWetBulb_AVL,DBRH_AVL,DBSky_AVL,DBWind_EWN,DBDryBulb_EWN,...,DBSky_RDU,Date,year,month,day,hour,weekday,season,EST_Holiday,non_working
0,2016-01-01 06:00:00+00:00,2016-01-01 01:00:00-05:00,5324.4385,16.0,42.0,40.0,82.0,0.0,6.0,57.0,...,100.0,2016-01-01,2016,1,1,1,Friday,winter,1,1
1,2016-01-01 07:00:00+00:00,2016-01-01 02:00:00-05:00,5137.2178,10.0,41.0,39.0,82.0,0.0,9.0,57.0,...,100.0,2016-01-01,2016,1,1,2,Friday,winter,1,1
2,2016-01-01 08:00:00+00:00,2016-01-01 03:00:00-05:00,4970.3784,9.0,41.0,39.0,79.0,0.0,9.0,57.0,...,80.0,2016-01-01,2016,1,1,3,Friday,winter,1,1
3,2016-01-01 09:00:00+00:00,2016-01-01 04:00:00-05:00,4881.2212,11.0,41.0,39.0,79.0,0.0,8.0,56.0,...,80.0,2016-01-01,2016,1,1,4,Friday,winter,1,1
4,2016-01-01 10:00:00+00:00,2016-01-01 05:00:00-05:00,4916.1343,9.0,40.0,38.0,83.0,0.0,7.0,56.0,...,100.0,2016-01-01,2016,1,1,5,Friday,winter,1,1


In [7]:
stations = ["AVL","EWN","FAY","FBG","GSB","ILM","NCA","NKT","POB","RDU"]
def calc_cdd(x):
    if x >= 65: # CDD
        return  x - 65 
    else:
        return 0
        
        
def calc_hdd(x):      
    if x<= 65:   # HDD
        return 65 - x
    else:
        return 0  
    
for i in stations:
    df['DBDryBulb_'+i+'_cdd']= df['DBDryBulb_'+i].apply(lambda x:calc_cdd(x))
    df['DBDryBulb_'+i+'_hdd']= df['DBDryBulb_'+i].apply(lambda x:calc_hdd(x))    
    

cdd_columns =  [i for i in df.columns if '_cdd' in i]    
hdd_columns =  [i for i in df.columns if '_hdd' in i]  

df.columns

Index(['Datetime_UTC', 'Datetime_EST', 'MWH_actual', 'DBWind_AVL',
       'DBDryBulb_AVL', 'DBWetBulb_AVL', 'DBRH_AVL', 'DBSky_AVL', 'DBWind_EWN',
       'DBDryBulb_EWN', 'DBWetBulb_EWN', 'DBRH_EWN', 'DBSky_EWN', 'DBWind_FAY',
       'DBDryBulb_FAY', 'DBWetBulb_FAY', 'DBRH_FAY', 'DBSky_FAY', 'DBWind_FBG',
       'DBDryBulb_FBG', 'DBWetBulb_FBG', 'DBRH_FBG', 'DBSky_FBG', 'DBWind_GSB',
       'DBDryBulb_GSB', 'DBWetBulb_GSB', 'DBRH_GSB', 'DBSky_GSB', 'DBWind_ILM',
       'DBDryBulb_ILM', 'DBWetBulb_ILM', 'DBRH_ILM', 'DBSky_ILM', 'DBWind_NCA',
       'DBDryBulb_NCA', 'DBWetBulb_NCA', 'DBRH_NCA', 'DBSky_NCA', 'DBWind_NKT',
       'DBDryBulb_NKT', 'DBWetBulb_NKT', 'DBRH_NKT', 'DBSky_NKT', 'DBWind_POB',
       'DBDryBulb_POB', 'DBWetBulb_POB', 'DBRH_POB', 'DBSky_POB', 'DBWind_RDU',
       'DBDryBulb_RDU', 'DBWetBulb_RDU', 'DBRH_RDU', 'DBSky_RDU', 'Date',
       'year', 'month', 'day', 'hour', 'weekday', 'season', 'EST_Holiday',
       'non_working', 'DBDryBulb_AVL_cdd', 'DBDryBulb_AVL_hdd',


In [9]:
project.save_data(file_name = "Actual_Load_Weather_Degree_Days V2.csv",data = df.to_csv(index=True))

{'file_name': 'Actual_Load_Weather_Degree_Days V2.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'ncemcpoc-donotdelete-pr-padhssbk4xmaam',
 'asset_id': '10b6e93f-245f-4e0b-b0eb-04e3b5cc9825'}